In [97]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

In [98]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# multi[curation_col]= multi[curation_col].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

# matches=not_tube
# matches[curation_col]=matches['height'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'7\/165','.44',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))


# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return f'{lower}" - {upper}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return f'{original_val}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

# matches[curation_col]=matches['Min'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [99]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]  

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

# str(round(num, 2))
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers

def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

In [100]:
customer_id = '90'
customer_name='%wbmason%'
buckets = """Multiple Buckets"""
strategy_version_input=682
attribute_id_input=2443
formatted_attribute = 'height'

In [101]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [102]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]

In [103]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df))
df = df[df['bucket_id'].isin(buckets)]
len(df)

5964


5964

In [107]:
df['buckets'].explode().value_counts()[0:15]

Protective Packing Materials             1544
Mailing & Packing Boxes                   975
Envelopes/Mailers                         773
Desks/Workstations/Workbenches            368
Table Replacement Parts & Accessories     318
Stools                                    279
Cabinets                                  175
Furniture Variety Packs                   140
Folding Tables & Tray Tables              127
Boxes, Baskets, & Bins                    127
Chairs                                    119
Utility Tables                            102
Temporary Seating                          95
Office Chairs                              92
Dividers/Partitions/Screens                81
Name: buckets, dtype: int64

In [122]:
mail=height[height['buckets'].astype(str)=='Mailing & Packing Boxes']
regex_pattern_tube=r'(?i)(\d+\.?\-?\s?\d*\/?\d*\"\s?h)|(?i)(\d+\.?\-?\s?\d*\/?\d*.?.?\s?x\s?\d+\.?\-?\s?\d*\/?\d*.?.?\s?x\s?\d+\.?\-?\s?\d*\/?\d*.?.?\s?)|()'
mail['matches']=mail['long_desc'].apply(lambda x: re_extract(regex_pattern_tube, x))
print(len(mail))
mail_df=mail[mail['matches'].astype(str)=='[]']
print(len(mail_df))

1004
634


In [126]:
regex_pattern_tube=r'(?i)(\d+\.?\-?\s?\d*\/?\d*\"\s?h)|(?i)(\d+\.?\-?\s?\d*\/?\d*.?.?\s?x\s?\d+\.?\-?\s?\d*\/?\d*)|()'
mail_df['matches']=mail_df['long_desc'].apply(lambda x: re_extract(regex_pattern_tube, x))
mail_df_double=mail_df[mail_df['matches'].astype(str)!='[]']
print(len(mail_df_double))
mail_df_double

601


product_id external_id  \
644    434461745  5637428202   
645    434461740  5637428197   
646    434461739  5637428196   
647    434461738  5637428195   
648    434461736  5637428193   
...          ...         ...   
7202   675184375  5637545265   
7217   675182519  5637545232   
7218   434457910  5637421570   
7267   704220905  5637549956   
7319   434462539  5637428381   

                                                                  product_name  \
644   W.B. Mason Co. Premium Telescoping Mailing Tubes, 3" x 36", Kraft, 24/CS   
645   W.B. Mason Co. Premium Telescoping Mailing Tubes, 3" x 30", Kraft, 24/CS   
646               W.B. Mason Co. Colored Mailing Tubes, 2" x 12", Black, 50/CS   
647     W.B. Mason Co. Premium Telescoping Mailing Tubes, 3" x 30", Red, 24/CS   
648   W.B. Mason Co. Premium Telescoping Mailing Tubes, 3" x 24", Kraft, 24/CS   
...                                                                        ...   
7202            W.B. Mason Co. Mailing Tubes with Caps, 4" x 48", Kraft, 15/CS   
7217              W.B. Mason Co. Colored Mailing Tubes, 3" x 36", Black, 24/CS   
7218         W.B. Mason Co. Triangle Mailing Tubes, 3" x 24 1/4", White, 50/BD   
7267    W.B. Mason Co. Corrugated Cube Shipping boxes, 20L x 20W x  20H, 20/BL   
7319          W.B. Mason Co. Crimped End Mailing Tubes, 3" x 30", Kraft, 24/CS   

                                                                                                                                                                                                                                                                                                                                                                                        long_desc  \
644   Premium Telescoping Mailing Tubes, 3" x 36", Kraft, 24/CS•Telescoping tubes provide a custom fit to your mailing•Telescoping tubes expand in length to accommodate different size documents with the same tube•"Heavy-duty, 1/8"", double wall, spiral wound construction"•"Tubes have a 6"" flush-joint cap"•To seal, simply adjust to desired length and seal with tape or shipping label   
645   Premium Telescoping Mailing Tubes, 3" x 30", Kraft, 24/CS•Telescoping tubes provide a custom fit to your mailing•Telescoping tubes expand in length to accommodate different size documents with the same tube•"Heavy-duty, 1/8"", double wall, spiral wound construction"•"Tubes have a 6"" flush-joint cap"•To seal, simply adjust to desired length and seal with tape or shipping label   
646                                                                             Colored Mailing Tubes, 2" x 12", Black, 50/CS•Attract attention with these bright, glossy, Colored Mailing Tubes•Tubes have a glossy finish•Use to identify or categorize different types of documents or mailings•Strong 3-ply spiral wound construction•Tubes seal with white plastic end caps included in case   
647     Premium Telescoping Mailing Tubes, 3" x 30", Red, 24/CS•Telescoping tubes provide a custom fit to your mailing•Telescoping tubes expand in length to accommodate different size documents with the same tube•"Heavy-duty, 1/8"", double wall, spiral wound construction"•"Tubes have a 6"" flush-joint cap"•To seal, simply adjust to desired length and seal with tape or shipping label   
648   Premium Telescoping Mailing Tubes, 3" x 24", Kraft, 24/CS•Telescoping tubes provide a custom fit to your mailing•Telescoping tubes expand in length to accommodate different size documents with the same tube•"Heavy-duty, 1/8"", double wall, spiral wound construction"•"Tubes have a 6"" flush-joint cap"•To seal, simply adjust to desired length and seal with tape or shipping label   
...                                                                                                                                                                                                                                                                                                           

In [105]:
regex_pattern=r'(?i)(\d+.?\d*\/\d*)|()'
height['matches']=height['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
total_na=height[height['matches'].astype(str)=='[]']
regex_pattern_total=r'(?i)(\d+\")|()'
total_na['new_matches']=total_na['long_desc'].apply(lambda x: re_extract(regex_pattern_total, x))
total_na_values=total_na[total_na['new_matches'].astype(str)=='[]']
print(len(total_na_values))

1360


In [94]:
regex_pattern_total=r'(?i)(\d+\")|()'
total_na['new_matches']=total_na['long_desc'].apply(lambda x: re_extract(regex_pattern_total, x))
total_na_values=total_na[total_na['new_matches'].astype(str)=='[]']
print(len(total_na_values))
total_na_values[curation_col]='n/a'

1360


# General

In [9]:
height=df
# regex_pattern_general=r'(?i)(\b\d+\.\d*.?\"\sH)|(\b\d+.?\"\sH)|(\b\d+\.\d*.?\"\sH)|(\d*\s?\-\d*\/\d*\\\"\sH)|(\d*\s\-?\d*\/\d*\\\"\sH)|(\d+\.?\d*''\sH)|()'
# # regex_pattern_general=r'(?i)(\b\d+\.\d*.?\"\s?H)|(\b\d+.?\"\s?H)|(\b\d+\.\d*.?\"\s?H)|(\d*\s?\-\d*\/\d*\\\"\s?H)|(\d*\s\-?\d*\/\d*\\\"\s?H)|(\d+\.?\d*''\s?H)|()'
# height['matches'] = height['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
# height['name_matches'] = height['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
# height['matches']= height['matches'].apply(lambda x: sorted(x, key=natural_key))
# print(len(height))
# print(len(height[height['matches'].astype(str)=='[]']))
# print(len(height[height['matches'].astype(str)!='[]']))

In [10]:
# # 3
# (\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|

# # All
# (?i)(\d\d\.?\d*\s?h)|(\d+\s?\d*\/\d*\s?h)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|(\d+\.?\s?\-?\d*\/?\d*"H)|(height\s?without\s?casters.?\s?\d+\")

# Protective Packing Materials    

In [11]:
packing=height[height['buckets'].astype(str)=='Protective Packing Materials']
len(packing)#variable

1685

In [12]:

# not_tube[['length', 'width', 'height']]=not_tube['matches'].str.split("x",expand=True,)

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

# not_tube['length']=not_tube['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# not_tube['width']=not_tube['width'].apply(lambda quote: quote.replace('"', ''))

# regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
# not_tube['length_new']=not_tube['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
# not_tube['width_new']=not_tube['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)

# not_tube['length_new'] = not_tube['length_new'].apply(lambda x: remove_fraction(x))
# not_tube['width_new'] = not_tube['width_new'].apply(lambda x: remove_fraction(x))

# not_tube['length_new'] = not_tube['length_new'].apply(lambda x: remove_frac(x))
# not_tube['width_new'] = not_tube['width_new'].apply(lambda x: remove_frac(x))

# not_tube['length_new']=not_tube['length_new'].astype(float)
# not_tube['width_new']=not_tube['width_new'].astype(float)

# not_tube['Max'] = not_tube[['length_new', 'width_new']].values.max(1)
# not_tube['Min'] = not_tube[['length_new', 'width_new']].values.min(1)


# TWOPLACES = Decimal(10) ** -2
# not_tube['Max']=not_tube['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x)))
# not_tube['Min']=not_tube['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

# envelope['envelope_matches']=envelope['envelope_matches'].str.get(0)
# envelope['envelope_matches']=envelope['envelope_matches'].apply(lambda x: x.lower())


# envelope[['length', 'width']]=envelope['envelope_matches'].str.split("x",expand=True,)

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

# envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# envelope['width']=envelope['width'].apply(lambda quote: quote.replace('"', ''))
# # envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('-',''))
# # envelope['width']=envelope['width'].apply(lambda quote: quote.replace('-',''))


# regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
# envelope['length_new']=envelope['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
# envelope['width_new']=envelope['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)


# envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_fraction(x))
# envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_fraction(x))

# envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_frac(x))
# envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_frac(x))

# Mailing & Packing Boxes

In [42]:
Boxes=height
Boxes=Boxes[Boxes['buckets'].astype(str)=='Mailing & Packing Boxes']
regex_pattern_tube=r'(?i)(tube)|()'
tubes=Boxes
tubes['tube_matches']=tubes['long_desc'].apply(lambda x: re_extract(regex_pattern_tube, x))

not_tube=tubes[tubes['tube_matches'].astype(str)=='[]']
regex_pattern=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?l\s?x\s?\d+.?w\s?x\s?\d+)|(\d+.?\d*\/?\d*\s?x\s?\d+.?\d*\/?\d*\s?x\s?\d+.?\d*\/?\d*)|()'              
not_tube['matches']=not_tube['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
not_tube=not_tube[not_tube['matches'].astype(str)!='[]']

In [43]:
regex_pattern=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?l\s?x\s?\d+.?w\s?x\s?\d+)|(\d+.?\d*\/?\d*\s?x\s?\d+.?\d*\/?\d*\s?x\s?\d+.?\d*\/?\d*)|()'              
tubes['matches']=tubes['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
tubes_new=tubes[tubes['matches'].astype(str)!='[]']

In [44]:
double_tubes=tubes[tubes['matches'].astype(str)=='[]']
regex_pattern_double=r'(?i)(\d+.?\d*\/?\d*.?\s?x\s?\d+.?\d*\/?\d*)|()'              
double_tubes['matches']=double_tubes['product_name'].apply(lambda x: re_extract(regex_pattern_double, x))

In [45]:
tubes_new['matches']=tubes_new['matches'].str.get(0).apply(lambda x: x.lower())
tubes_new[['length', 'width', 'height']]=tubes_new['matches'].str.split("x",expand=True,)

In [47]:
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r'7 200','7',str(measure)))
# not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'1232','12',str(measure)))
# not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'15250','15',str(measure)))
# not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'24141680','2',str(measure)))
# not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'16200','16',str(measure)))
# not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'9275','9',str(measure)))
# not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'1832','18',str(measure)))

In [48]:
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))

In [49]:
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
tubes_new['height']=tubes_new['height'].replace('"', '').apply(lambda quote: quote.replace('"', ''))

In [56]:
regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
tubes_new['height_new'] = tubes_new['height'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

In [58]:
tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_frac(x))
tubes_new['height_new']=tubes_new['height_new'].astype(float)
TWOPLACES = Decimal(10) ** -2
tubes_new['height_new']=tubes_new['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [59]:
tubes_new[curation_col]=tubes_new['height_new']
tubes_new[curation_col]=tubes_new[curation_col].apply(lambda x: str(x)+' in')

In [65]:
double_tubes=double_tubes[double_tubes['matches'].astype(str)!='[]']

In [127]:
# double_tubes[0:500]

In [71]:
double_tubes['matches']=double_tubes['matches'].str.get(0).apply(lambda x: x.lower())
double_tubes[['height','length']]=double_tubes['matches'].str.split("x",expand=True,)

ValueError: Columns must be same length as key

In [72]:
double_tubes['height']=double_tubes['height'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
double_tubes['height']=double_tubes['height'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))

KeyError: 'height'

In [60]:
double_tubes['height']=double_tubes['height'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
double_tubes['height']=double_tubes['height'].replace('"', '').apply(lambda quote: quote.replace('"', ''))

332


# Envelope

In [16]:
envelope=height[height['buckets'].astype(str)=='Envelopes/Mailers']
regex_pattern_general=r'(?i)(roll)|()'
envelope['envelope_matches']=envelope['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
envelope=envelope[envelope['envelope_matches'].astype(str)=='[]']

In [17]:
regex_pattern=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|()'              
envelope['matches']=envelope['product_name'].apply(lambda x: re_extract(regex_pattern, x))
envelope=envelope[envelope['matches'].astype(str)=='[]']

In [18]:
# envelope[0:500] # Waiting on Arv and Ross

In [19]:
regex_pattern_general=r'(?i)(\d+\s?\-?\d+\/\d+.?\s?x\s?\d+\s?\-?\d+\/\d+)|(\d+\.\d*.?\s?x\s?\d+\.\d*)|(\d+\s?\-?\d+\/\d+.?\s?x\s?\d+\.?\d*)|(\d+\s?.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+)|(\d+.?\d*.?\s?x\s?\d+.?\d*)|(\d+\.?\s?\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|()'    
envelope['envelope_matches']=envelope['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
envelope=envelope[envelope['envelope_matches'].astype(str)!='[]']

In [20]:
# def array(df):
#     df['matches_one']=df['envelope_matches'].apply(lambda x : np.array(x)[0])
#     try:
#         df['matches_two']=df['envelope_matches'].apply(lambda x : np.array(x)[1])
#     except:
#         df['matches_two']='n/a'

In [21]:
# def drop_characters(data):
#     data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))

In [22]:
# array(envelope)

In [23]:
# envelope['match_one'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[0])

In [24]:
# try:
#     envelope['matches'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[1])
# except:
#     envelope['matches'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[0])

In [25]:
# double=envelope[envelope['external_id'].astype(str)=='5637503643']
# double=envelope[0:1]
# double

In [26]:
# double['n_matches']=double['envelope_matches'].apply(lambda x : np.array(x)[0])
# # double['m_matches']=double['envelope_matches'].apply(lambda x : np.array(x)[1])
# double

In [27]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# double= double.apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [28]:
# double=double.apply(lambda x: f'"{x}"')
# double=double.apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# double[['one','two']]=double['envelope_matches'].str.split(",",expand=True,)
# double=double.apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# double

In [33]:
envelope

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, envelope_matches, matches]
Index: []

In [29]:
envelope['envelope_matches']=envelope['envelope_matches'].str.get(0)
envelope['envelope_matches']=envelope['envelope_matches'].apply(lambda x: x.lower())

In [30]:
envelope[['length', 'width']]=envelope['envelope_matches'].str.split("x",expand=True,)

ValueError: Columns must be same length as key

In [31]:
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'77-34','34',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 500\/','12',str(measure)))
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'12 500\/','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 2500\/','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 1000\/','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12\,100','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'10 100/','10',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'8 100/','8',str(measure)))

envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12100','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12\,100','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'10 100/','10',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'8 100/','8',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'16 100','16',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'19 500','19',str(measure)))

KeyError: 'length'

In [32]:
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

envelope['length']=envelope['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

KeyError: 'length'

In [ ]:
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
envelope['width']=envelope['width'].apply(lambda quote: quote.replace('"', ''))

regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
envelope['length_new']=envelope['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
envelope['width_new']=envelope['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)

envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_fraction(x))
envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_fraction(x))

envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_frac(x))
envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_frac(x))

In [ ]:
# envelope['length_new']=envelope['length_new'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# envelope['width_new']=envelope['width_new'].apply(lambda quote: quote.replace('"', ''))

In [ ]:
TWOPLACES = Decimal(10) ** -2
envelope['length_new']=envelope['length_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
envelope['width_new']=envelope['width_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
envelope[curation_col]=envelope['width_new']
envelope[curation_col]=envelope[curation_col].apply(lambda x: str(x)+' in')

In [ ]:
len(envelope)

In [ ]:
envelope

# Office Chairs

In [34]:
# oc=height[height['buckets'].astype(str)=='Office Chairs']
# # regex_pattern_general=r'(?i)(roll)|()'
# regex_pattern=r'(?i)(?i)(\d+.?\d*\/?\d*.?\s?to\s?[a-z\s]{0,20}\d*.?\d*\/?\d*)|(seat.?height.?\s?\d+\"\s?\-?\s?\d*\s?\d*\/?\d*)|(\d\d\d?\d*\".{0,3}\d\d\.\d*\"\s?H)||()'              
# oc['matches']=oc['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# oc_na=oc[oc['matches'].astype(str)=='[]']
# oc=oc[oc['matches'].astype(str)!='[]']
# oc_na[curation_col]=r'n/a'

In [36]:
# len(oc_na) 

# Desk WorkBenches

In [37]:
# desk=height[height['buckets'].astype(str)=='Desks/Workstations/Workbenches']
# # regex_pattern_general=r'(?i)(roll)|()'
# regex_pattern=r'(?i)(?i)(height)|("H)|(\d\d\.?\d\s?h)|(\d+\s?\d*\/\d*\s?h)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|(\d+.?\d*\"\s?To\s?\d+")|()'              
# desk['matches']=desk['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# desk_na=desk[desk['matches'].astype(str)=='[]']
# desk_na[curation_col]=r'n/a'

In [38]:
# len(desk_na) 

# Chairs

In [47]:
# chair=height[height['buckets'].astype(str)=='Chairs']
# regex_pattern=r'(?i)(\d+\.?\s?\d*\/?\d*\"\s?h)|()'              
# chair['matches']=chair['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# chair_height=chair[chair['matches'].astype(str)!='[]']

In [48]:
# chair_three=chair[chair['matches'].astype(str)=='[]']
# regex_pattern_three=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|()'              
# chair_three['matches']=chair_three['long_desc'].apply(lambda x: re_extract(regex_pattern_three, x))

In [49]:
# chair_na=chair_three[chair_three['matches'].astype(str)=='[]']
# chair_na[curation_col]=r'n/a'

In [39]:
# print(len(chair_na)) 

# Drawers

In [40]:
drawer=height[height['buckets'].astype(str)=='Drawers']
print(len(drawer))
regex_pattern_drawer=r'(?i)(\d\d\.?\d*\s?h)|(\d+\s?\d*\/\d*\s?h)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|(\d+.?\d*\"\s?To\s?\d+")|(\d+\.?\s?\-?\d*\/?\d*"H)|(height\s?without\s?casters.?\s?\d+\")|(height)|()'                           
drawer['matches']=drawer['long_desc'].apply(lambda x: re_extract(regex_pattern_drawer, x))
drawer_na=drawer[drawer['matches'].astype(str)=='[]']
drawer_na[curation_col]=r'n/a'
print(len(drawer_na))

268
0


In [41]:
drawer_next=drawer[drawer['matches'].astype(str)!='[]']
regex_pattern_short=r'(?i)(\d\d\.?\d*\s?h)|(\d+\s?\-?\d*\/\d*\s?h)|(\d+\.?\s?\-?\d*\/?\d*"H)|(height\s?without\s?casters.?\s?\d+\")|()'                           
drawer_next['new_matches']=drawer_next['long_desc'].apply(lambda x: re_extract(regex_pattern_short, x))
drawer_short=drawer_next[drawer_next['new_matches'].astype(str)!='[]']
print(len(drawer_short))

225


In [42]:
drawer_short['height']=drawer_short['new_matches'].str.get(0)
drawer_short['height']=drawer_short['height'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
drawer_short['height']=drawer_short['height'].apply(lambda measure: re.sub(r"\'|,|:",'',str(measure)))
drawer_short['height']=drawer_short['height'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
drawer_short['height']=drawer_short['height'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
drawer_short['height'] = drawer_short['height'].apply(lambda x: remove_fraction(x))
drawer_short['height'] = drawer_short['height'].apply(lambda x: remove_frac(x))
drawer_short['height']=drawer_short['height'].astype(float)
TWOPLACES = Decimal(10) ** -2
drawer_short['height']=drawer_short['height'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [43]:
drawer_short[curation_col]=drawer_short['height']
drawer_short[curation_col]=drawer_short[curation_col].apply(lambda x: str(x)+' in')

In [44]:
drawer_three=drawer_next[drawer_next['new_matches'].astype(str)=='[]']
regex_pattern_three=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|()'
drawer_three['new_matches']=drawer_three['long_desc'].apply(lambda x: re_extract(regex_pattern_three, x))
drawer_three=drawer_three[drawer_three['new_matches'].astype(str)!='[]']

In [45]:
drawer_three['new']=drawer_three['new_matches'].str.get(0)
drawer_three[['length', 'width', 'height']]=drawer_three['new'].str.split("x",expand=True,)
drawer_three=drawer_three[(drawer_three['external_id'].astype(str)!='5637192984')&(drawer_three['external_id'].astype(str)!='5637192989')&(drawer_three['external_id'].astype(str)!='5637192983')&(drawer_three['external_id'].astype(str)!='5637192989')&(drawer_three['external_id'].astype(str)!='5637192990')]                      

In [46]:
drawer_three['height']=drawer_three['height'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
drawer_three['height']=drawer_three['height'].apply(lambda measure: re.sub(r"\'|,|:",'',str(measure)))
drawer_three['height']=drawer_three['height'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
drawer_three['height']=drawer_three['height'].replace('"', '').apply(lambda quote: quote.replace('"', ''))

In [47]:
regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
drawer_three['height_new']=drawer_three['height'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
drawer_three['height_new'] = drawer_three['height_new'].apply(lambda x: remove_fraction(x))

In [48]:
drawer_three['height_new']=drawer_three['height_new'].apply(lambda x: remove_fraction(x))
drawer_three['height_new']=drawer_three['height_new'].apply(lambda x: remove_frac(x))
drawer_three['height_new']=drawer_three['height_new'].astype(float)
TWOPLACES = Decimal(10) ** -2
drawer_three['height_new']=drawer_three['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
drawer_three[curation_col]=drawer_three['height_new']

In [49]:
drawer_three[curation_col]=drawer_three[curation_col].apply(lambda x:str(x)+' in')

In [64]:
print(len(drawer_short))

225


In [65]:
drawer_short

product_id external_id  \
600     434422198  5637353989   
964     434393907  5637247370   
1015    434495040  5637468154   
1489    434442154  5637389481   
1611    434477951  5637145808   
1612    434477947  5637145800   
1613    434477946  5637145799   
1765   1175474102  5637575277   
1766   1175474101  5637575276   
1767   1175474100  5637575275   
1768   1175474099  5637575274   
1801    434408237  5637305028   
1861   1175355794  5637574818   
1869   1175355795  5637574819   
1870    434496870  5637472697   
1960    434481436  5637146114   
1961    434481435  5637146113   
1962    434481433  5637146111   
2028   1175355787  5637574298   
2320    434385352  5637220285   
2321    434385351  5637220283   
2586    434443805  5637396022   
3015    434489500  5637159501   
3016    434489497  5637159498   
3907    434415790  5637328031   
4018    434381470  5637175537   
4203    434406659  5637304522   
4204    434406658  5637304521   
4205    434406657  5637304520   
4206    434406654  5637304517   
4207    434406653  5637304516   
4208    434406651  5637304514   
4209    434406649  5637304512   
4210    434406648  5637304511   
4211    434406647  5637304510   
4212    434406643  5637304506   
4278    434497722  5637473095   
4333    434382190  5637190535   
4461    434488958  5637171607   
4674    434475043  5637454709   
4675    434475039  5637454705   
4676    434475032  5637454698   
4677    434475028  5637454694   
4678    434475026  5637454692   
4679    434475021  5637454687   
4699    434415036  5637328263   
4837   1175477216  5637575947   
4926    434438510  5637381909   
5038    434393741  5637245692   
5382   1175502584  5637574855   
5383   1175398997  5637574851   
5389    434450445  5637408585   
5390    434450441  5637408578   
5395    434450431  5637408567   
5396    434450430  5637408566   
5407    434450425  5637408562   
5450    434501470  5637473426   
5451    434501468  5637473424   
5462   1175424285  5637577939   
5463   1175424284  5637577938   
5511    434391896  5637246095   
5512    434391890  5637246089   
5514    434391886  5637246085   
5556    434489498  5637159499   
5652    434393904  5637247367   
5708    434477956  5637145816   
5709    434477955  5637145815   
5710    434477948  5637145801   
5736    434481430  5637146108   
5863    434445192  5637398201   
5898    434381467  5637175531   
5980    434449619  5637408636   
5987   1175502585  5637574856   
6003   1175446416  5637574372   
6049   1175477212  5637575943   
6071    434445191  5637398200   
6087    434496871  5637472698   
6089    434480428  5637147860   
6092    434438509  5637381908   
6104   1175446413  5637574370   
6105    434488946  5637159552   
6106    434477943  5637145792   
6107    434391894  5637246093   
6108    434406644  5637304507   
6122    434496929  5637473203   
6134   1175355793  5637574817   
6137    434391887  5637246086   
6138    434449624  5637408642   
6176   1175408154  5637574333   
6177    434477953  5637145814   
6202   1175477214  5637575945   
6260    434385919  5637218703   
6478   1175446418  5637574374   
6497    434387354  5637218171   
6541    434406674  5637304544   
6552    434406660  5637304523   
6553    434406656  5637304519   
6555    434406645  5637304508   
6557    434407632  5637306803   
7044    434449620  5637408637   
7270    434488967  5637171616   
7291    434450457  5637408599   
7292    434450451  5637408598   
7293    434450453  5637408595   
7294    434450432  5637408568   
7617    434488942  5637159529   
7618    434488941  5637159526   
7619    434488939  5637159506   
7620    434488935  5637159502   
7621    434489495  5637159496   
7727    434386605  5637218389   
7954    434393910  5637247375   
7983    434391897  5637246096   
7984    434391891  5637246090   
7985    434391889  5637246088   
8056    434442159  5637389486   
8245    434450444  5637408584   
8246    434495041  5637468155   
8264    434386654  5637218662   
8284    434409142  5637305527   
8286   1175424

In [57]:
# print(len(drawer_na))

In [54]:
# print(len(chair_na)) 

In [55]:
# len(desk_na) 

In [56]:
# len(oc_na) 

In [58]:
len(envelope)

5576

In [60]:
envelope[0:500]

product_id external_id  \
23     434497246  5637497860   
24     434497245  5637497859   
25     434497784  5637497856   
26     434497779  5637497851   
27     434519159  5637497850   
28     434519158  5637497849   
29     434511675  5637497825   
31     434497789  5637497868   
32     434497788  5637497867   
33     434497787  5637497866   
34     434497249  5637497863   
35     434497247  5637497861   
63     434500366  5637480082   
82     434514523  5637497378   
83     434515977  5637497369   
84     434516005  5637497366   
85     434516004  5637497365   
114    434466076  5637435550   
119    434468784  5637435503   
121    444420825  5637504150   
122    444420822  5637504148   
123    444420821  5637504147   
124    444420820  5637504146   
125    444420815  5637504142   
126    444423088  5637504138   
127    444423087  5637504137   
128    444423086  5637504136   
129    444423084  5637504134   
130    444423083  5637504133   
172    434461980  5637429322   
173    434461978  5637429320   
174    434461977  5637429319   
175    434461974  5637429316   
176    434461971  5637429313   
177    434461969  5637429311   
178    434461965  5637429307   
179    434461961  5637429302   
180    434461955  5637429296   
181    434461953  5637429294   
182    434464681  5637429290   
183    434464678  5637429287   
184    434464675  5637429284   
185    434464674  5637429283   
186    434464673  5637429282   
187    434464664  5637429273   
191    434439276  5637383145   
192    434439275  5637383144   
193    434436877  5637383139   
194    434436876  5637383137   
195    434436872  5637383133   
196    434436868  5637383127   
197    434436867  5637383126   
198    434436865  5637383124   
199    434436863  5637383122   
200    434436861  5637383120   
201    434436859  5637383118   
202    434436858  5637383117   
203    434436856  5637383115   
204    434436855  5637383114   
205    434436854  5637383113   
214    434468863  5637435598   
215    529179161  5637519670   
216    529179160  5637519669   
217    529179158  5637519667   
218    529192742  5637519666   
219    529192739  5637519663   
220    529179157  5637519656   
221    529179156  5637519655   
222    529179155  5637519654   
223    529179153  5637519652   
224    529177646  5637519647   
225    529177645  5637519646   
226    529177644  5637519645   
227    529177642  5637519643   
228    529179146  5637519637   
229    434458576  5637421765   
231    529179145  5637519636   
232    529192776  5637519635   
233    529192773  5637519632   
234    529192771  5637519631   
235    529192769  5637519629   
236    529192767  5637519627   
237    529177636  5637519625   
238    529177633  5637519622   
239    529177632  5637519621   
240    529177631  5637519620   
293    453796589  5637503817   
294    453796588  5637503816   
335    434515209  5637496376   
336    434515206  5637496374   
337    434511577  5637496370   
338    434515205  5637496365   
339    434436766  5637382143   
340    434515498  5637496264   
341    434515472  5637496239   
342    434515470  5637496237   
343    434515468  5637496235   
349    434516443  5637494124   
350    434516429  5637494117   
351    434493715  5637172921   
352    434516411  5637494099   
353    434516407  5637494087   
354    434516405  5637494086   
355    434516399  5637494080   
356    434516398  5637494079   
357    434516393  5637494075   
358    434516392  5637494074   
359    434516386  5637494068   
360    434516385  5637494067   
361    434516381  5637494063   
362    434516379  5637494061   
363    434516378  5637494060   
364    434516374  5637494056   
365    434516373  5637494055   
366    434493712  5637172918   
367    434493711  5637172917   
368    434493710  5637172916   
369    434511070  5637494046   
370    434511068  5637494044   
371    434511066  5637494042   
372    434511063  5637494040   
373    434511061  5637494038   
374    434511059  5637494036   
375    434493708  5637172914   
376   

In [57]:
err

NameError: name 'err' is not defined

In [ ]:
matches=height[height['matches'].astype(str)!='[]']
matches[curation_col]=height['matches'].astype(str)
matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" H',' in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'H','in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'Inch',' in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'12"','12 in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'in',' in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))

In [ ]:
matches=matches[(matches['external_id'].astype(str)!='69592262')&(matches['external_id'].astype(str)!='65277614')
       &(matches['external_id'].astype(str)!='47401167')
       &(matches['external_id'].astype(str)!='69671877')
       &(matches['external_id'].astype(str)!='41637418')
       &(matches['external_id'].astype(str)!='65277607')
       &(matches['external_id'].astype(str)!='41971185')
       &(matches['external_id'].astype(str)!='65977835')
       &(matches['external_id'].astype(str)!='47001480')
       &(matches['external_id'].astype(str)!='61470569')
       &(matches['external_id'].astype(str)!='47444829')
       &(matches['external_id'].astype(str)!='44283988')
       &(matches['external_id'].astype(str)!='69689885')
       &(matches['external_id'].astype(str)!='60988829')
       &(matches['external_id'].astype(str)!='63571479')
       &(matches['external_id'].astype(str)!='17604490')
       &(matches['external_id'].astype(str)!='61470576')
       &(matches['external_id'].astype(str)!='41637333')]
len(matches)

In [ ]:
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"101\/2",'10.5',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"325\/8",'32.68',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"17\.0",'17',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\-7\/8",'.88',str(measure)))

matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\-3\/4",'.75',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\-1\\2",'.5',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"1\\5",'0.2',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\-1\/4",'.25',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"91/2",'9.5',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"1/2",'.5',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\-",'',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"3/4",'.75',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"1/5",'0.2',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"1/4",'0.25',str(measure)))
# matches[curation_col].explode().value_counts()

In [ ]:
matches

In [ ]:
na_upload=height[height['matches'].astype(str)=='[]']
regex_pattern_na=r'(\d*\-Inch)|()'
na_upload['matches'] = na_upload['product_name'].apply(lambda x: re_extract(regex_pattern_na, x))
na_upload=na_upload[na_upload['matches'].astype(str)=='[]']
# height['matches']= height['matches'].apply(lambda x: sorted(x, key=natural_key))
len(na_upload)

# foot is pulling in inches with two single quotes

In [ ]:
foot=na_upload
regex_pattern_ft=r'(\d*\'\s?H)|()'
foot['matches'] = foot['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
foot=foot[foot['matches'].astype(str)!='[]']
foot.head(1)

In [ ]:
matches_ft=foot[foot['matches'].astype(str)!='[]']
matches_ft[curation_col]=foot['matches'].astype(str)
matches_ft[curation_col]=matches_ft[curation_col].apply(lambda x: f'"{x}"')
matches_ft[curation_col]=matches_ft[curation_col].apply(lambda measure: re.sub(r"\'\'",'"',str(measure)))
matches_ft[curation_col]=matches_ft[curation_col].apply(lambda measure: re.sub(r"\' H",' ft',str(measure)))

matches_ft[curation_col]=matches_ft[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches_ft[curation_col]=matches_ft[curation_col].apply(lambda measure: re.sub(r'ft',' ft',str(measure)))
matches_ft[curation_col]=matches_ft[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches_ft.head(1)

In [ ]:
na_upload
regex_pattern_ft=r'(\d*\'\s?H)|()'
na_upload['matches'] = na_upload['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
na_upload=na_upload[na_upload['matches'].astype(str)=='[]']
len(na_upload)

In [ ]:
na_upload[na_upload['buckets'].astype(str)!='TV Stands/Media Consoles'][0:500]

# Seperate Bukets and start curating

# Wall Sculptures

In [ ]:
wall=df[df['buckets'].astype(str)=='Wall Sculptures & Dimensional Art']
regex_pattern_wall = r"(?i)(\d*\.?\d*.?Inch\s?x\s?\d*\.?\d*.?Inch)|(\d*.?Inch)|(\d*\.\d*.?Inch)|()" 
wall['matches'] =wall['long_desc'].apply(lambda x: re_extract(regex_pattern_wall, x))
wall['name_matches'] =wall['product_name'].apply(lambda x: re_extract(regex_pattern_wall, x))
wall_df=wall[wall['name_matches'].astype(str)!='[]']

# Square

In [ ]:
wall_df['name_matches']=wall_df['name_matches'].str.get(0)
wall_df[['length', 'width']]=wall_df['name_matches'].str.split("x",expand=True,)
wall_df.head(1)

wall_df['length']=wall_df['length'].apply(lambda measure: re.sub(r"\-Inch",'',str(measure)))
wall_df['width']=wall_df['width'].apply(lambda measure: re.sub(r"\-Inch",'',str(measure)))  
wall_df['length']=wall_df['length'].apply(lambda measure: re.sub(r" ",'',str(measure)))
wall_df['width']=wall_df['width'].apply(lambda measure: re.sub(r" ",'',str(measure))) 
wall_df['length']=wall_df['length'].astype(str)
wall_df['width']=wall_df['width'].astype(str)

wall_df['compared']=wall_df['length'].equals(wall_df['width'])
wall_df.loc[(wall_df['length']==wall_df['width']), 'compared']=wall_df['length']
wall_new=wall_df[wall_df['compared'].astype(str)!='False']
wall_new[curation_col]=wall_new['compared']   
wall_new[curation_col]=wall_new[curation_col].apply(lambda x: x+' in')
wall_new.head(1)

square=wall_df[wall_df['width'].astype(str)=='None']
square[curation_col]=square['length']
square[curation_col]=square[curation_col].apply(lambda x: x+' in')
square.head(1)

In [ ]:
len(wall_new)

In [ ]:
square=wall_df[wall_df['width'].astype(str)=='None']
square[curation_col]=square['length']
square[curation_col]=square[curation_col].apply(lambda x: x+' in')
square.head(1)

In [ ]:
len(square)

# Trees

In [ ]:
trees=df[df['buckets'].astype(str)=='Trees, Shrubs, Topiaries, & Other Plants']
# \d*.?Foot
regex_pattern_tree = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|()" 
trees['matches'] =trees['long_desc'].apply(lambda x: re_extract(regex_pattern_tree, x))
trees['name_matches'] =trees['product_name'].apply(lambda x: re_extract(regex_pattern_tree, x))
trees_df=trees[trees['name_matches'].astype(str)!='[]']

In [ ]:
#import regex as re
matches_tree=trees_df
matches_tree[curation_col]=matches_tree['name_matches'].astype(str)
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda x: f'"{x}"')
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\-Foot",' ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'\-foot',' ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\-1\/2",'.5',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r' 1\/2','.5',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r" Foot",' ft',str(measure)))

matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'7 ft 6\-Inch','7.5 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"6 ft 5\-Inch",'6.4 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'4 ft 6\-Inch','4.6 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"7 ft 5\-Inch",'7.4 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"2 ft 5\-Inch",'2.4 ft',str(measure)))

matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches_tree.head(10)

In [ ]:
matches_tree[curation_col].explode().value_counts()

In [ ]:
len(matches_tree)

# Stools

In [ ]:
stools=df[df['buckets'].astype(str)=='Stools']
regex_pattern_stool = r"(?i)(\d*\.\d*.?Inch)|(\d*.?Inch)|(\d*.?.?H)|()" 
stools['matches'] =stools['long_desc'].apply(lambda x: re_extract(regex_pattern_stool, x))
stools['name_matches'] =stools['product_name'].apply(lambda x: re_extract(regex_pattern_stool, x))
stools_df=stools[stools['name_matches'].astype(str)!='[]']

In [ ]:
#import regex as re
matches_stool=stools_df
matches_stool[curation_col]=matches_stool['name_matches'].astype(str)
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda x: f'"{x}"')
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r"\-Inch",' in',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r" Inch",' in',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches_stool.head(10)

In [ ]:
matches_stool[curation_col].explode().value_counts()

In [ ]:
len(matches_stool)

# Side Tables

In [ ]:
side_tables=df[df['buckets'].astype(str)=='Side/End Tables']
regex_pattern_table = r"(?i)(\d*\.\d*”.?H)|(\d*\.\d*.?Inch)|(\d*.?Inch)|(\d*”.?H)|()" 
side_tables['matches'] =side_tables['long_desc'].apply(lambda x: re_extract(regex_pattern_table, x))
side_tables['name_matches'] =side_tables['product_name'].apply(lambda x: re_extract(regex_pattern_table, x))
side_tables_df=side_tables[side_tables['matches'].astype(str)!='[]']
side_tables_df['matches']= side_tables_df['matches'].apply(lambda x: sorted(x, key=natural_key))

In [ ]:
#import regex as re
matches_side_table=side_tables_df
matches_side_table[curation_col]=side_tables_df['matches'].astype(str)
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda x: f'"{x}"')
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'\” H',' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'\”H',' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r" ",'',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r"in",' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))

In [ ]:
len(matches_side_table)

# Sculptures and figurines

In [ ]:
sculptures=df[df['buckets'].astype(str)=='Sculptures, Figurines, & Decorative Objects']
# \d*.?Foot
regex_pattern_sc = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|(\d*\.\d*”.?H)|(\d*\.\d*.?Inch.?H)|(\d*.?Inch.?H)|(\d*”.?H)|()" 
sculptures['matches'] =sculptures['long_desc'].apply(lambda x: re_extract(regex_pattern_sc, x))
sculptures['name_matches'] =sculptures['product_name'].apply(lambda x: re_extract(regex_pattern_sc, x))
sculptures_df=sculptures[(sculptures['matches'].astype(str)!='[]')]
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='69241383']
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='16623601']
sculptures_df['matches']= sculptures_df['matches'].apply(lambda x: sorted(x, key=natural_key))
# sculptures_df

In [ ]:
regex_pattern_sc_mis = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|(\d*\.\d*''.?H)|(\d*\.\d*.?Inch.?H)|(\d*.?Inch.?H)|(\d*''.?H)|()" 
sculptures['mis_matches'] =sculptures['long_desc'].apply(lambda x: re_extract(regex_pattern_sc_mis, x))
sculptures['misc_name_matches'] =sculptures['product_name'].apply(lambda x: re_extract(regex_pattern_sc_mis, x))
sculptures_df=sculptures[(sculptures['misc_name_matches'].astype(str)!='[]')]
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='69241383']
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='16623601']
sculptures_df['matches']= sculptures_df['matches'].apply(lambda x: sorted(x, key=natural_key))
sculptures_df

In [ ]:
#import regex as re
matches_sc=sculptures_df
matches_sc[curation_col]=sculptures_df['matches'].astype(str)

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda x: f'"{x}"')
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'\”','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'H',' in',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'  ',' ',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'Inch H',' in',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'Inch','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"\-",'',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"in",' in',str(measure)))

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'1 in","16 in','16 in',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"6 ft 5\-Inch",'6.4 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'4 ft 6\-Inch','4.6 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"7 ft 5\-Inch",'7.4 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"2 ft 5\-Inch",'2.4 ft',str(measure)))

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_sc.head(10)

In [ ]:
matches_sc[curation_col].explode().value_counts()

In [ ]:
matches_sc[matches_sc['matches'].astype(str)==' foot']

In [ ]:
len(matches_sc)

# foot

In [ ]:
# regex_pattern = r"(?i)(\b\d+\.\d*.?\"\s?H\s)|(\b\d+.?\"\s?H)|(\b\d+\.\d*.?\"\s?H)|(\d*\s?\-\d*\/\d*\\\"\s?H)|(\d*\s\-?\d*\/\d*\\\"\s?H)|()" 
# regex_pattern_diam = r'(?i)(\d+\.\d*\s?\\\"\s?i?n?\s?diam)|(\d*\s?\\\"\s?i?n?\s?diam)|()'
# regex_pattern_foot = r"(?i)(\b\d+\.\d*\'\s?H\s)|(\b\d+.?\'\s?H)|(\b\d+\.\d*\'\s?H)|(\d*\s?\-\d*\/\d*\\?\'\s?H)|(\d*\s\-?\d*\/\d*\\\'\s?H)|()" 

In [ ]:
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# df['diam_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diam, x))
# df['foot_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_foot, x))
# len(df)

In [ ]:
# foot=df[df['foot_matches'].astype(str)!='[]']
# foot=foot[foot['buckets'].astype(str)=='Trees, Shrubs, Topiaries, & Other Plants']
# len(foot)

In [ ]:
# # x=df[df['matches'].astype(str)!='[]']
# x=df
# len(x)
# # len(x)

In [ ]:
# x=x[x['matches'].astype(str)!='[]']

In [ ]:
# replacement_foot = {
#     "\' H": " ft",
#     "' H": "ft",
#     '  ft': " ft",
#     "\'H": " ft",
#     "' h": " ft",
#     "3\' H": "3 ft"
# }

In [ ]:
# # Replacement
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: [replace_all(string, replacement_foot) for string in x] if type(x)==list else x)

In [ ]:
# # Replacement
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: [replace_all(string, replacement_foot) for string in x] if type(x)==list else x)
# # foot.head(3)

In [ ]:
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# # foot.head(3)

In [ ]:
# #import regex as re
# matches_foot=foot
# matches_foot[curation_col]=matches_foot['foot_matches'].astype(str)
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda x: f'"{x}"')
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_foot.head(3)

In [ ]:
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r"\-1\/2",'.5',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\"\\xa0H',' ft',str(measure)))

In [ ]:
# matches_foot[curation_col].explode().value_counts()

In [ ]:
replacement = {
    '\" H ': " in",
    "\ in": " in",
    '\" H': " in",
    '\" H': " in",
    '\" H ': " in",
    "\ in": " in",
    '\" H': " in",
    '\" H': " in",
    '\" h': " in",
    '\"H': " in"
}

In [ ]:
x=x[x['matches'].astype(str)!='[]']
x.head(1)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(1)

In [ ]:
x['matches']= x['matches'].apply(
    lambda x: sorted(x, key=natural_key))
x.head(1)

In [ ]:
# x=x[(x['matches'].astype(str)!='3/4 in')&(x['matches'].astype(str)!='1/4 in')&(x['matches'].astype(str)!='1/5 in')&(x['matches'].astype(str)!=' 3/4 in')]

In [ ]:
# replacement_diam = {
#     'diam': "",
#     " ": "",
#     '\"': "",
#     'in': "",
#     'Diam': ""
# }
# # Replacement
# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: [replace_all(string, replacement_diam) for string in x] if type(x)==list else x)

# # Replacement
# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: [replace_all(string, replacement_diam) for string in x] if type(x)==list else x)
# diam.head(3)

# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# diam.head(3)

# matches

In [ ]:
# matches=x

In [ ]:
# #import regex as re
# matches[curation_col]=matches['matches'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches.head(3)

In [ ]:
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1\/2','.5',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3\/4','.75',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3\/4','.75',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1\/2','.5',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1\/4','.25',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1\/4','.25',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3\/8','.38',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3\/8','.38',str(measure)))

# matches=matches[matches[curation_col].astype(str)!='["3/4 in"]']
# matches=matches[matches[curation_col].astype(str)!='["1/4 in"]']
# matches=matches[matches[curation_col].astype(str)!='["1/5 in"]']

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-4\/5','.8',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 4\/5','.8',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/5','.2',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/5','.2',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-5/8','.62',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 5/8','.62',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-2/3','.66',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 2/3','.66',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3/10','.3',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3/10','.3',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/3','.33',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/3','.33',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-8/9','.89',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 8/9','.89',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3/5','.6',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3/5','.6',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-2/5','.4',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 2/5','.4',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-7/10','.7',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 7/10','.7',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/10','.1',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/10','.1',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/8','.13',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/8','.13',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-7/8','.88',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 7/8','.88',str(measure)))

In [ ]:
# # matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'1575 in','15.75 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\xa0','',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'4563 in','45.63 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'3625 in','36.25 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.625','.63',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.375','.38',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.875','.88',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'335 in","635 in','33.5 in","63.5 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'45 in","395 in','45 in","39.5 in',str(measure)))

In [ ]:
# matches[matches[curation_col].astype(str)=='["0.59 in"]']

In [ ]:
# matches[matches[curation_col].astype(str)=='["45 in","395 in"]']

In [ ]:
len(matches)

In [ ]:
# matches_diam=diam

In [ ]:
# #import regex as re
# matches_diam[curation_col]=matches_diam['diam_matches'].astype(str)
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda x: f'"{x}"')
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_diam.head(3)

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\\\\',' in',str(measure)))

In [ ]:
# matches_diam=matches_diam[matches_diam['buckets'].astype(str)!='Trees, Shrubs, Topiaries, & Other Plants']
# matches_diam=matches_diam[matches_diam['buckets'].astype(str)=='Hanging Accents & Accessories']

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\\',' in',str(measure)))

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'.875','.88',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\"10 in\"\,\"10 in\"','"10 in"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'.725','.73',str(measure)))

In [ ]:
# matches_diam[curation_col].explode().value_counts()[0:500]

In [ ]:
# error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

In [ ]:
len(envelope) print(len(drawer_three)) print(len(drawer_short))

# send to the folder for upload

In [44]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets- {attribute}-- Height-envelope upload_Shawn.csv',index=False)

In [45]:
looks_good('Bed Bath & Beyond', attribute, df, envelope)

In [95]:
def looks_good_na(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-NA - {attribute}-- total_na_values upload_Shawn.csv',index=False)

In [96]:
looks_good_na('Bed Bath & Beyond', attribute, df, total_na_values)